# task 1

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define transformations for the training and testing data
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 for AlexNet compatibility
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load the MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# Load pre-trained ResNet18 and modify the first and final layers
import torchvision.models as models

resnet18 = models.resnet18(pretrained=False)
resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
resnet18.fc = nn.Linear(resnet18.fc.in_features, 10)

# Load pre-trained AlexNet and modify the first and final layers
alexnet = models.alexnet(pretrained=False)
alexnet.features[0] = nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2)
alexnet.classifier[6] = nn.Linear(alexnet.classifier[6].in_features, 10)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

resnet18 = resnet18.to(device)
alexnet = alexnet.to(device)
criterion = nn.CrossEntropyLoss()

# Optimizers
resnet18_optimizer = optim.Adam(resnet18.parameters(), lr=0.001)
alexnet_optimizer = optim.Adam(alexnet.parameters(), lr=0.001)

def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    return model

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Train ResNet18
print("Training ResNet18...")
resnet18 = train_model(resnet18, train_loader, criterion, resnet18_optimizer)

# Train AlexNet
print("Training AlexNet...")
alexnet = train_model(alexnet, train_loader, criterion, alexnet_optimizer)

# Evaluate the models
resnet18_accuracy = evaluate_model(resnet18, test_loader)
alexnet_accuracy = evaluate_model(alexnet, test_loader)

print(f'ResNet18 Accuracy: {resnet18_accuracy:.2f}%')
print(f'AlexNet Accuracy: {alexnet_accuracy:.2f}%')


Training ResNet18...
Epoch [1/10], Loss: 0.0977
Epoch [2/10], Loss: 0.0422
Epoch [3/10], Loss: 0.0316
Epoch [4/10], Loss: 0.0259
Epoch [5/10], Loss: 0.0239
Epoch [6/10], Loss: 0.0207
Epoch [7/10], Loss: 0.0176
Epoch [8/10], Loss: 0.0157
Epoch [9/10], Loss: 0.0134
Epoch [10/10], Loss: 0.0102
Training AlexNet...


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch [1/10], Loss: 0.3748
Epoch [2/10], Loss: 0.0941
Epoch [3/10], Loss: 0.0792
Epoch [4/10], Loss: 0.0675
Epoch [5/10], Loss: 0.0637
Epoch [6/10], Loss: 0.0551
Epoch [7/10], Loss: 0.0556
Epoch [8/10], Loss: 0.0530
